## Deliverable 2: Create a Customer Travel Destinations Map

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key


In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/gnrkaur/Desktop/Development/Module6/World_Weather_Analysis/Weather_Database/cities.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.15,-70.92,55.40,54,75,25.28,broken clouds
1,1,Hualmay,PE,-11.10,-77.61,71.19,69,52,10.78,broken clouds
2,2,Arraial Do Cabo,BR,-22.97,-42.03,82.40,69,0,26.40,clear sky
3,3,Dikson,RU,73.51,80.55,31.41,99,100,27.11,light snow
4,4,Nemuro,JP,43.32,145.57,33.51,88,23,8.99,few clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"]<= max_temp) & 
    (city_data_df["Max Temp"]>= min_temp)]


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Arraial Do Cabo,BR,-22.97,-42.03,82.40,69,0,26.40,clear sky
6,6,Saint-Pierre,RE,-21.34,55.48,75.20,73,40,2.24,scattered clouds
9,9,Gabu,GW,12.28,-14.22,79.72,23,60,3.00,broken clouds
11,11,Samarai,PG,-10.62,150.67,81.14,83,68,8.50,broken clouds
13,13,Avarua,CK,-21.21,-159.78,80.60,88,82,5.82,moderate rain


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Arraial Do Cabo,BR,82.40,clear sky,-22.97,-42.03,
6,Saint-Pierre,RE,75.20,scattered clouds,-21.34,55.48,
9,Gabu,GW,79.72,broken clouds,12.28,-14.22,
11,Samarai,PG,81.14,broken clouds,-10.62,150.67,
13,Avarua,CK,80.60,moderate rain,-21.21,-159.78,
33,Manado,ID,75.20,light rain,1.49,124.85,
37,Saint-Philippe,RE,75.20,scattered clouds,-21.36,55.77,
39,Mandiana,GN,80.62,overcast clouds,10.63,-8.68,
43,Pimentel,PE,75.20,clear sky,-6.84,-79.93,
44,Moree,AU,77.00,overcast clouds,-29.47,149.85,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found .... Skipping")
        

Hotel not found .... Skipping
Hotel not found .... Skipping
Hotel not found .... Skipping
Hotel not found .... Skipping
Hotel not found .... Skipping
Hotel not found .... Skipping
Hotel not found .... Skipping
Hotel not found .... Skipping
Hotel not found .... Skipping
Hotel not found .... Skipping
Hotel not found .... Skipping
Hotel not found .... Skipping
Hotel not found .... Skipping
Hotel not found .... Skipping


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != '']
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Arraial Do Cabo,BR,82.40,clear sky,-22.97,-42.03,Pousada Porto Praia
6,Saint-Pierre,RE,75.20,scattered clouds,-21.34,55.48,Lindsey Hôtel
9,Gabu,GW,79.72,broken clouds,12.28,-14.22,Hotel karsa
11,Samarai,PG,81.14,broken clouds,-10.62,150.67,Nuli Sapi
13,Avarua,CK,80.60,moderate rain,-21.21,-159.78,Paradise Inn
...,...,...,...,...,...,...,...
650,Porbandar,IN,76.12,clear sky,21.64,69.61,Lords Inn Porbandar
652,Xuddur,SO,78.39,clear sky,4.12,43.89,Banjanay Hotel
655,Dhidhdhoo,MV,83.03,few clouds,6.88,73.10,Dhidhoo beach
659,Vianopolis,BR,75.56,broken clouds,-16.74,-48.52,Hotel Regente


In [13]:
# 8a. Create the output File (CSV)
output_data_file_2 = "hotels.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file_2, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))